**Importando bibliotecas**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

**Criando uma aplicação Spark**

In [ ]:
spark = SparkSession.builder.master("local").appName("Classificação").getOrCreate()

**Importando dados**

In [ ]:
churn = spark.read.csv("/home/rowedder/download/churn.csv", inferSchema=True, header=True, sep=";")

In [ ]:
churn.show(3)

**Criando o objeto (RFormula)**

In [ ]:
formula = RFormula(formula="Exited ~ .", featuresCol="features", labelCol="label", handleInvalid="skip")

In [ ]:
churn_trans = formula.fit(churn).transform(churn).select("features", "label")

In [ ]:
churn_trans.show(truncate=False)

**Dividindo os dados em porcentagem para treino e teste**

In [ ]:
churn_treino, churn_teste = churn_trans.randomSplit([0.7,0.3])

In [ ]:
print(churn_treino.count())

In [ ]:
print(churn_teste.count())

**Criando o objeto (Árvore de Decisão)**

In [ ]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

**Criando o modelo de treino (fit)**

In [ ]:
modelo = dt.fit(churn_treino)

**Testando o modelo (Previsão)**

In [ ]:
previsao = modelo.transform(churn_teste)

In [ ]:
previsao.show()

**Avaliando a performance do modelo de árvore de decisão (Métrica de Performance)**

In [ ]:
avaliar = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")

In [ ]:
areaUnderRoc = avaliar.evaluate(previsao)

In [ ]:
print(areaUnderRoc)